In [1]:
import os
import numpy as np
import tensorflow as tf
!pip install --upgrade tensorflow
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

2024-04-12 09:17:03.974759: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-12 09:17:03.974862: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-12 09:17:04.094047: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 4.7 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 22.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.1
    Uninstalling tensorboard-2.15.1:
      Successfully uninstalled tensorboard-2.15.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer,

In [2]:
def load_images_from_directory(directory):
    image_data = []
    labels = []
    classes = os.listdir(directory)
    class_to_index = {class_name: i for i, class_name in enumerate(classes)}  # Assign numeric labels
    for class_name in classes:
        class_dir = os.path.join(directory, class_name)
        for image_file in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_file)
            img = load_img(image_path, target_size=(224, 224))  # ResNet50 input size
            img_array = img_to_array(img)
            img_array = img_array / 255.0  # Normalize pixel values
            image_data.append(img_array)
            labels.append(class_to_index[class_name])  # Convert class name to numeric label
    return np.array(image_data), np.array(labels)

train_data, train_labels = load_images_from_directory('/kaggle/input/autism-dataset/dataset/train')
validation_data, validation_labels = load_images_from_directory('/kaggle/input/autism-dataset/dataset/valid')

In [8]:
base_model = ResNet50(weights='imagenet', include_top=False)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
# Add additional dense layers
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.25)(x)  # Add dropout for regularization
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
# Final Classification layer
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [9]:
model.compile(optimizer=Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [12]:
model.fit(train_data, train_labels, 
          epochs=20, 
          steps_per_epoch = 100, 
          validation_data=(validation_data, validation_labels))

Epoch 1/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 22s 130ms/step - accuracy: 0.7764 - loss: 0.4600 - val_accuracy: 0.5300 - val_loss: 2.4195
Epoch 2/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.7773 - loss: 0.4515 - val_accuracy: 0.5900 - val_loss: 1.2170
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.7887 - loss: 0.4572 - val_accuracy: 0.5800 - val_loss: 1.3197
Epoch 4/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.7792 - loss: 0.4745 - val_accuracy: 0.6400 - val_loss: 0.7602
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.7834 - loss: 0.4528 - val_accuracy: 0.6200 - val_loss: 1.2795
Epoch 6/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.7942 - loss: 0.4291 - val_accuracy: 0.5400 - val_loss: 1.5520
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.7816 - loss: 0.4511 - val_accuracy: 0.7200 - val_loss: 0.9074
Epoch 8/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.7926 - loss: 0.4287 - val_ac

In [13]:
model.save('from_scratch.keras')

In [14]:
# Load and preprocess test data
test_data = []
test_filenames = []
test_dir = '/kaggle/input/autism-dataset/dataset/test'

for image_file in os.listdir(test_dir):
    image_path = os.path.join(test_dir, image_file)
    img = load_img(image_path, target_size=(224, 224))  # ResNet50 input size
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalize pixel values
    test_data.append(img_array)
    test_filenames.append(image_file)

test_data = np.array(test_data)

# Make predictions
predictions = model.predict(test_data)

# Since you don't have ground truth labels, you can save the predictions along with the filenames
# For example, you can print the filename and corresponding predicted class probabilities
# for filename, prediction in zip(test_filenames, predictions):
#     print(f'Filename: {filename}, Predictions: {prediction}')


 5/10 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step

W0000 00:00:1712914569.862712      91 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 447ms/step
